In [19]:
import pandas as pd
import numpy as np

In [20]:
# Load data from CSV
df = pd.read_csv("../data/Building_Violations_sample_50000.csv")


In [21]:
## Clean data

# Delete useless variable
del df['Unnamed: 0']

# Convert column headers to snake case
import re

def spaces_to_snake(str_with_spaces):
    s = re.sub('\s+','_',str_with_spaces)
    return s.lower()

df.columns = [spaces_to_snake(col) for col in df.columns]
df.columns

# Delete SSA column
del df['ssa']

# Convert dates to datetime format
df.violation_date = pd.to_datetime(df.violation_date)
df.violation_status_date = pd.to_datetime(df.violation_status_date)
df.violation_last_modified_date = pd.to_datetime(df.violation_last_modified_date)

In [22]:
# Write CSV with cleaned data
df.to_csv("../data/Building_Violations_sample_50000_clean.csv")

In [14]:
df.dtypes

id                                       int64
violation_last_modified_date    datetime64[ns]
violation_date                  datetime64[ns]
violation_code                          object
violation_status                        object
violation_status_date           datetime64[ns]
violation_description                   object
violation_location                      object
violation_inspector_comments            object
violation_ordinance                     object
inspector_id                            object
inspection_number                        int64
inspection_status                       object
inspection_waived                       object
inspection_category                     object
department_bureau                       object
address                                 object
property_group                           int64
latitude                               float64
longitude                              float64
location                                object
dtype: object

In [15]:
# Check how many different violation codes there are
df.violation_code.describe()

count        50000
unique         757
top       CN190019
freq          2463
Name: violation_code, dtype: object

In [27]:
df.violation_description.value_counts()

ARRANGE PREMISE INSPECTION        2463
POST OWNER/MANAGERS NAME/#        1619
REPAIR EXTERIOR WALL              1488
REPAIR PORCH SYSTEM               1326
VACANT BUILDING - REGISTER        1065
REPLCE WINDOW PANES, PLEXGLAS     1011
PLANS & PERMITS REQ - CONTRCTR     964
REPAIR EXTERIOR STAIR              937
FILE BLDG REGISTRATION             904
INSTALL SMOKE DETECTORS            892
MAINTAIN OR REPAIR ELECT ELEVA     730
DEBRIS, EXCESSIVE                  725
STOP/REMOVE NUISANCE               696
CARB MONOX DETECT IN RESID         638
OBSTRUCTIONS IN EXIT WAY           625
...
CARB MONOX DETECT IN HOTEL        1
ELIMINATE TRIPPING HAZARD PASS    1
ENCL/SECURE OPENINGS PASS         1
REP/REPL HANDRAIL FOR ESCL        1
IDENTIFY ELE MACHINES             1
PRV CONDENSER                     1
REP/REPL DEF HALL BUTTONS PASS    1
HEATER 8FT ABOVE FLR IN GARAGE    1
PROVIDE HOT WATER                 1
PRV ABSORBER                      1
VERIFICATIONS OF D.U.             1
NONCOMPLIANT MA

In [52]:
# Create a list of violation codes <--> violation descriptions
list_of_violations = df[['violation_code', 'violation_description']].sort(['violation_code', 'violation_description']).drop_duplicates()

In [58]:
list_of_violations.to_csv('../data documentation/list_of_violations.csv',index=False)

In [28]:
# Check how many kinds of violation status
df.violation_status.describe()


count     50000
unique        3
top        OPEN
freq      36355
Name: violation_status, dtype: object

In [29]:
df.violation_status.value_counts()

OPEN        36355
COMPLIED    13500
NO ENTRY      145
dtype: int64